<a href="https://colab.research.google.com/github/Maximi652/OpenMeter_Analysis/blob/main/qwen3_4b_WANDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Vorbereitung in Google Colab
A) Qwen3-4B benötigt die aktuelle transformers-Version, sonst gibt es einen KeyError: 'qwen3'. Nimm transformers >= 4.51.0.

In [1]:
!nvidia-smi

# Frisches Python-Env in Colab
!pip -q install "torch>=2.2" "transformers>=4.51.0" "accelerate>=0.33.0" datasets safetensors evaluate sentencepiece

Sun Aug 17 07:41:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

B) Repo & Daten holen (BioASQ-Ordner oder Fallback HF)


In [2]:
# Euer Repo klonen (nur falls noch nicht da)
!git clone -q https://github.com/Maximi652/efficient-slm-architectures.git
%cd efficient-slm-architectures

# Falls ihr bereits eigene JSON/CSV im Data/-Ordner habt, nützt die:
import os, glob, json

data_dir = "Data"
use_local = os.path.isdir(data_dir) and len(glob.glob(os.path.join(data_dir, "*"))) > 0
print("Nutze lokale BioASQ-Daten:", use_local)

if not use_local:
    from datasets import load_dataset
    # kleines, öffentliches Task-B-Subset (für Kalibrierung & PPL reicht das)
    ds = load_dataset("bigbio/bioasq_task_b", name="bioasq_task_b_source", split="train")
    # wir extrahieren Texte (Frage + Snippets) als Rohtext
    def to_text(example):
        q = example.get("question", "")
        ctxs = example.get("snippets", []) or []
        ctx_txt = " ".join(s.get("text","") for s in ctxs) if isinstance(ctxs, list) else ""
        return {"text": f"Question: {q}\nContext: {ctx_txt}\nAnswer:"}
    ds_text = ds.map(to_text, remove_columns=ds.column_names)
else:
    # Minimal-Parser: alle .txt/.json mit Text zusammensetzen
    texts = []
    for fp in glob.glob(os.path.join(data_dir, "**/*.*"), recursive=True):
        if fp.endswith(".txt"):
            texts.append({"text": open(fp, "r", encoding="utf-8", errors="ignore").read()})
        elif fp.endswith(".json"):
            try:
                obj = json.load(open(fp, "r", encoding="utf-8", errors="ignore"))
                if isinstance(obj, dict) and "text" in obj: texts.append({"text": obj["text"]})
                elif isinstance(obj, list):
                    for o in obj:
                        if isinstance(o, dict) and "text" in o: texts.append({"text": o["text"]})
            except Exception: pass
    from datasets import Dataset
    ds_text = Dataset.from_list(texts)
print(ds_text[:2])


/content/efficient-slm-architectures
Nutze lokale BioASQ-Daten: True
{}


2) Qwen3-4B laden (Base-Variante) & Tokenizer vorbereiten
Qwen3-4B (4.0B Parameter, 36 Layer, GQA) ist als CausalLM in HF verfügbar. Für Pruning empfiehlt sich Base statt Instruct. Achte auf die empfohlene Transformers-Version.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, math
torch.backends.cuda.matmul.allow_tf32 = True

model_id = "Qwen/Qwen3-4B-Base"  # Base ist für Pruning am saubersten
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,     # fp16 genügt auf T4/L4/A100
    device_map="auto"              # schiebt automatisch auf GPU
).eval()

# Chat-Template nicht nötig; wir tokenisieren reinen Text


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

3) Wanda in „reinem“ PyTorch/Transformers (modellagnostisch)
Wanda-Idee in einem Satz: Wir berechnen pro nn.Linear die mittlere L1-Eingangsaktivierung je Input-Kanal auf einem kleinen Kalibrier-Corpus und wichten damit die Gewichtsmagnituden
Die kleinsten Scores werden auf 0 gesetzt (Zielsparsität pro Layer oder global).


A) Kalibrierungs-Aktivierungen sammeln
Wir hooken alle nn.linear - Embeddings/LM-Head lassen wir aus.

In [4]:
from collections import defaultdict
import torch.nn as nn
from datasets import Dataset

# 1) kleine Kalibrierungsauswahl
calib_size = 200  # 100-500 reicht i.d.R. für 30-60% Sparsity stabil
if len(ds_text) > calib_size:
    ds_calib = ds_text.select(range(calib_size))
else:
    ds_calib = ds_text

# 2) Hooks: mittlere |x| je Input-Kanal für alle Linear-Layer sammeln
act_sums = {}
act_counts = {}
linear_layers = []

def register_linear_hooks(model):
    for name, mod in model.named_modules():
        if isinstance(mod, nn.Linear):
            if any(k in name for k in ["embed_tokens","lm_head"]):  # skip
                continue
            linear_layers.append((name, mod))
            act_sums[name] = torch.zeros(mod.in_features, dtype=torch.float32, device=model.device)
            act_counts[name] = 0

            def hook_fn(mod, inp, out, layer_name=name):
                x = inp[0]  # [*, in_features]
                # normiere auf 2D
                x2 = x.detach().to(torch.float32).reshape(-1, x.shape[-1]).abs()
                act_sums[layer_name] += x2.sum(dim=0)
                act_counts[layer_name] += x2.shape[0]
            mod.register_forward_hook(hook_fn)

register_linear_hooks(model)

# 3) Kalibrier-Forward-Pass
max_len = 512
with torch.no_grad():
    for ex in ds_calib:
        text = ex["text"] if isinstance(ex, dict) else ex
        enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
        _ = model(**enc)

# 4) Aktivierungsgewichte g_j = E[|x_j|]
act_means = {name: (act_sums[name] / max(1, act_counts[name])) for name, _ in linear_layers}


B) Wanda-Scores berechnen & prunen
Wir prunen pro Layer eine feste Quote (50%). Optional: "globales" Thresholding über alle Layer. Achtung: Norm-Layer, Embeddings und lm_head bleiben unberührt (sonst kippt PPL schnell).

In [5]:
import torch.nn.utils.prune as prune

target_sparsity = 0.5     # 50% unstrukturiert als Startpunkt (30–60% üblich)
layerwise = True          # pro Layer Quote; auf False für globales Thresholding

# 1) Scores je Layer
scores = {}
for name, mod in linear_layers:
    W = mod.weight.detach().to(torch.float32)                      # [out, in]
    g = act_means[name].clamp(min=1e-8)                            # [in]
    # Broadcast: |W| * g (per Input-Kanal)
    S = W.abs() * g.unsqueeze(0)                                   # [out, in]
    scores[name] = S

# 2a) Layerweise Schwellwerte bestimmen
if layerwise:
    thresholds = {}
    for name, mod in linear_layers:
        S = scores[name].flatten()
        k = int(target_sparsity * S.numel())
        thresh = torch.topk(S, k, largest=False).values.max() if k>0 else torch.tensor(-1., device=S.device)
        thresholds[name] = float(thresh)
else:
    # 2b) Globaler Schwellwert
    allS = torch.cat([scores[n].flatten() for n,_ in linear_layers])
    k = int(target_sparsity * allS.numel())
    global_thresh = torch.topk(allS, k, largest=False).values.max() if k>0 else torch.tensor(-1., device=allS.device)

# 3) Masken anwenden
for name, mod in linear_layers:
    S = scores[name]
    if layerwise:
        mask = (S > thresholds[name]).to(mod.weight.dtype)
    else:
        mask = (S > global_thresh).to(mod.weight.dtype)
    # in original-Gewichtsform
    mask = mask.to(mod.weight.device)
    # echtes „Hard“-Pruning (keine reparam), damit Speichern einfach ist
    with torch.no_grad():
        mod.weight.mul_(mask)

# 4) Statistiken
total_w = 0
total_zero = 0
for name, mod in linear_layers:
    W = mod.weight.detach()
    total_w += W.numel()
    total_zero += (W==0).sum().item()
print(f"Globale Sparsity: {total_zero/total_w:.3f}")


Globale Sparsity: 0.501


4) Pruned-Modell speichern (safetensors) & wieder laden

In [6]:
from safetensors.torch import save_file
import os, shutil

save_dir = f"Qwen3-4B-Base-wanda-{int(target_sparsity*100)}"
os.makedirs(save_dir, exist_ok=True)

# Wichtig: auf CPU verschieben, bevor wir speichern (kleinerer Peak-VM)
state = {k: v.detach().to("cpu") for k, v in model.state_dict().items()}
save_file(state, os.path.join(save_dir, "model.safetensors"))
# Config & Tokenizer mitspeichern
model.config.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Gespeichert nach:", save_dir)

# Test: Laden
from transformers import AutoModelForCausalLM, AutoTokenizer
tok2 = AutoTokenizer.from_pretrained(save_dir)
mod2 = AutoModelForCausalLM.from_pretrained(save_dir, torch_dtype=torch.float16, device_map="auto").eval()


Gespeichert nach: Qwen3-4B-Base-wanda-50
